# The random phase approximation (RPA)


TO BE WRITTEN : (material below)


- Recall the formula for RPA


- TODO : 
 
 - we grab chi(q,omega)
-------------------------------------------


The Lindhardt susceptibility $\chi_0$ is the exeact susceptibility for the non-interacting case $U=0$, however for finite interactions the susceptibility of the system $\chi$ is given by the Behte-Salpeter equation

$$ \chi = \chi_0 + \chi_0 \Gamma \chi $$

where $\Gamma$ is the particle-hole irreducible vertex function, containing all diagrams with insertions of the interaction that can not be separated by cutting a pair of particle-hole single-particle propagators $G G$.

The first order contribution to the vertext $\Gamma$ is the bare interaction $U$ and the approximation

$$ \Gamma \approx U $$

gives the so-called random phase approximation for $\chi$, i.e.

$$ \chi_{RPA} = \chi_0 + \chi_0 U \chi_{RPA} $$

Rewriting this equation gives $\chi_{RPA}$ as

$$ \chi_{RPA} = \frac{\chi_0}{1 - U \chi_0} $$

we note that the denominator of this equation can in general go to zero, whereby the susceptibility $\chi_{RPA}$ diverges. Whence already the RPA approximation can be used to compute instabilities of the system towards, e.g., anti-ferromagnetic symmetry breaking.

As an example we compute $\chi_{RPA}$ for the square lattice and the enhancement of the $k = (\pi, \pi)$ peak as a function of $U$.

## Load Lindhard chi from archive

In [ ]:
# Pick up chi from HDF5
# RENAME in chi_RPA ?
# Q : write this
# Q : plot chi
# Q : find the instability
from pytriqs.archive import HDFArchive
from pytriqs.gf import Gf, Idx, inverse

with HDFArchive("tpsc.h5",'r') as R:
    chi0_kw = R['chi0_kw']

## Compute RPA susceptibility (this is not really necessary)

One can also compute it on the fly (see below)

In [ ]:
# The DOUBLE loop will be faster...
def chi_rpa_from_U_and_chi0_wk(chi0_wk, U):
    chi_rpa = chi0_kw.copy()
    kmesh = chi0_kw.mesh[0]
    wmesh = chi0_kw.mesh[1]
    for k in kmesh:
        for w in wmesh:
            chi_rpa[k,w] = chi0_kw[k,w] / (1. - U * chi0_kw[k,w])
#            chi_rpa[k,:] << inverse(1. - U * chi0_kw[k,:]) * chi0_kw[k,:]
    return chi_rpa

chi_rpa = chi_rpa_from_U_and_chi0_wk(chi0_wk, 4)

## Plot the susceptibility

In [ ]:
from pytriqs.plot.mpl_interface import plt
from pytriqs.gf import inverse

nk = chi_rpa.mesh[0].linear_dims[0]+1
k = np.linspace(0, 2*np.pi, nk, endpoint=True)
kx, ky = np.meshgrid(k, k)

chi = lambda kx, ky: chi_rpa([kx,ky,0],0).real

# Note that I can avoid the cell above by just doing
#U = 4
#chi = lambda kx, ky: (chi0_kw([kx,ky,0],0) / (1. - U * chi0_kw([kx,ky,0],0))).real

# 2d plot
plt.figure(figsize=(3.25*2, 5))
plt.title('Static RPA susceptibility $\chi_\mathrm{RPA}(k, \omega=0)$')
plt.pcolor(kx, ky, np.vectorize(chi)(kx,ky))
plt.xlim(0, 2*np.pi)
plt.ylim(0, 2*np.pi)
plt.xticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])    
plt.yticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])
plt.colorbar()
plt.tight_layout()
plt.xlabel(r'$k_x$')
plt.ylabel(r'$k_y$')


# Here is a 3d plot. With "matplotlib inline" one cannot interact
from mpl_toolkits.mplot3d import axes3d, Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(kx, ky, np.vectorize(chi)(kx,ky), cmap="jet", rstride=8, cstride=8)
ax.set_xlabel(r"$k_x$")
ax.set_ylabel(r"$k_y$")
ax.set_zlabel(r"$\chi_\mathrm{RPA}$")


## Plot along some path for different values of $U$ (TODO)

In [ ]:
from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    chi0_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
chi0_plot = interp(k_path_vecs[:, :2])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')

plt.plot(k_plot, chi0_plot.real, label=r'$\chi_0$')

U_vec = np.arange(0.2, 1.2, 0.2)

for U in U_vec:
    print 'U =', U
    chi_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, U)
    chi = lambda kx, ky: 
    
    k = np.linspace(0, 2*np.pi, kmesh.linear_dims[0]+1, endpoint=True)


    plt.plot(k, chi_plot.real, label=r'$\chi_{RPA}$, $U=%2.2f$' % U)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.legend(loc='best')
plt.savefig('figure_chi0_k_bandpath.pdf')

At some critical $U_c$ the susceptibility diverges $\chi \rightarrow \infty$ within the random phase approximation. To determine $U_c$ we can study the root of the inverse susceptibility $\chi_{RPA}^{-1}$.

For the square lattice it is sufficient to study the response at $k_{AF}= (\pi, \pi)$ since it is the point where the response diverges. Analytically this occurs when the denominator is zero $1 - U_c \chi_0(0, k_{AF}) = 0$, i.e.

$$ U_c^{(RPA)} = \frac{1}{\chi_0(0, k_{AF})} $$

numerically this looks like

## Find critical $U$

In [ ]:

chi_inv = lambda U: 1. - U * chi0_kw([np.pi, np.pi, 0],0).real

U_c = 1 / chi0_kw([np.pi,np.pi,0], 0).real

Ur = np.linspace(0,5,100)


plt.figure(figsize=(3.25*3, 8))
plt.title("Determination of critical $U$ in RPA")
plt.plot(Ur, chi_inv(Ur), label=r'$\chi_{RPA}^{-1}$')
plt.plot(Ur, np.zeros_like(Ur), 'k', lw=0.5)
plt.plot(U_c, 0, 'rs', label=r'$U_c \approx %2.2f$'%U_c, alpha=0.5)
plt.legend(loc='best')
plt.xlabel(r'$U$')
plt.ylabel(r'$\chi_\mathrm{RPA}^{-1}$')

